<a href="https://colab.research.google.com/github/TanyamonSiri/Machine_Learning_Project/blob/main/Recommedation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommendation System Coursework

## Import Library

In [1]:
import pandas as pd
import numpy as np

## Load dataset directly from kaggle

Cigarette Rating by Smokers: https://www.kaggle.com/datasets/mikhailverghese/cigarette-reviews-by-smokers


In [2]:
# make directory name ".kaggle"
! mkdir ~/.kaggle
# copy the ".kaggle.json" into new directory
! cp kaggle.json ~/.kaggle/
# Allocate the required permission for this file
! chmod 600 ~/.kaggle/kaggle.json

In [3]:
# download file from kaggle
! kaggle datasets download mikhailverghese/cigarette-reviews-by-smokers

  0% 0.00/65.4k [00:00<?, ?B/s]
100% 65.4k/65.4k [00:00<00:00, 65.8MB/s]


In [4]:
# unzip file
! unzip /content/cigarette-reviews-by-smokers.zip

Archive:  /content/cigarette-reviews-by-smokers.zip
  inflating: smokerdata.csv          


In [5]:
# remove unused file
! rm /content/cigarette-reviews-by-smokers.zip
! rm /content/kaggle.json

In [42]:
# create dataframe
df = pd.read_csv('/content/smokerdata.csv')
df

,User,Brand,Variety,Type,Date,Strength,Taste,Price,Rating
0,ghanta,Marlboro,Reds,Regular,2007-06-21,Medium,Tolerable,Fair,3
1,Karawasa,Marlboro,Reds,Regular,2007-06-23,Very Strong,Tolerable,Fair,4
2,Tintedace,Camel,Turkish Gold,Regular,2007-06-23,Medium,Very Poor,High,1
3,Karawasa,Camel,Turkish Gold,Regular,2007-06-23,Strong,Pleasant,Fair,4
4,Tintedace,Newport,Full Flavor,Regular,2007-06-23,Medium,Very Pleasant,Fair,5
...,...,...,...,...,...,...,...,...,...
5433,sahrah120s,Marlboro,Full Flavor,100s,2018-11-18,Medium,Pleasant,Fair,4
5434,sahrah120s,Marlboro,Black,100s,2018-11-18,Strong,Pleasant,High,4
5435,floras,Benson and Hedges,Gold,Regular,2019-01-24,Medium,Very Pleasant,Very High,5
5436,Monijonnlopez,Marlboro,Special Blend (Red),100s,2019-01-26,Medium,Very Pleasant,Very High,5


## Exploratory Data Analysis (EDA)

In [7]:
df.isnull().sum() # no missing value

User        0
Brand       0
Variety     0
Type        0
Date        0
Strength    0
Taste       0
Price       0
Rating      0
dtype: int64

In [8]:
df.shape

(5438, 9)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5438 entries, 0 to 5437
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   User      5438 non-null   object
 1   Brand     5438 non-null   object
 2   Variety   5438 non-null   object
 3   Type      5438 non-null   object
 4   Date      5438 non-null   object
 5   Strength  5438 non-null   object
 6   Taste     5438 non-null   object
 7   Price     5438 non-null   object
 8   Rating    5438 non-null   int64 
dtypes: int64(1), object(8)
memory usage: 382.5+ KB


In [10]:
df.describe()

,Rating
count,5438.000000
mean,3.929386
std,1.206847
min,1.000000
25%,3.000000
50%,4.000000
75%,5.000000
max,5.000000


In [11]:
df.nunique() # one user may rate multiple brands

User        2953
Brand         24
Variety       56
Type           6
Date        2188
Strength       5
Taste          5
Price          5
Rating         5
dtype: int64

In [12]:
# number of each rated brand in dataset
df.Brand.value_counts()

Marlboro             1654
Camel                 877
Pall Mall             527
Newport               475
American Spirit       393
KOOL                  157
L&M                   149
Lucky Strike          143
Benson and Hedges     131
Winston               121
Parliament            121
Djarum                116
Virginia Slims        103
Maverick               86
Pyramid                60
USA Gold               59
305's                  54
Nat Sherman            54
Sonoma                 30
Dunhill                30
Salem                  26
marlboro               26
Basic                  25
Misty                  21
Name: Brand, dtype: int64

In [13]:
df.columns

Index(['User', 'Brand', 'Variety', 'Type', 'Date', 'Strength', 'Taste',
       'Price', 'Rating'],
      dtype='object')

## Data Preprocessing

In order to create effective recommendation system for customer, filter only product with a rating of 5.

In [14]:
df_rating_5 = df.query('Rating == 5')
df_rating_5

,User,Brand,Variety,Type,Date,Strength,Taste,Price,Rating
4,Tintedace,Newport,Full Flavor,Regular,2007-06-23,Medium,Very Pleasant,Fair,5
11,Lothar,American Spirit,Mediums,Regular,2007-06-23,Strong,Very Pleasant,High,5
16,Grimey,Camel,Lights,Regular,2007-06-23,Medium,Very Pleasant,Low,5
27,Joe Camel,Camel,Wides Filters,Regular,2007-06-24,Strong,Pleasant,Fair,5
43,raytheon,Lucky Strike,Filters,Regular,2007-06-28,Medium,Pleasant,High,5
...,...,...,...,...,...,...,...,...,...
5429,Heike,Benson and Hedges,Full Flavor,100s,2018-08-14,Strong,Very Pleasant,High,5
5431,marlbororob89,Marlboro,Reds,100s,2018-08-20,Strong,Very Pleasant,Fair,5
5435,floras,Benson and Hedges,Gold,Regular,2019-01-24,Medium,Very Pleasant,Very High,5
5436,Monijonnlopez,Marlboro,Special Blend (Red),100s,2019-01-26,Medium,Very Pleasant,Very High,5


In [15]:
# Brand
df_rating_5.Brand.value_counts()

Marlboro             649
Camel                322
Pall Mall            247
Newport              238
American Spirit      163
Lucky Strike         111
Virginia Slims        80
Benson and Hedges     68
KOOL                  64
Djarum                53
L&M                   48
Parliament            47
Nat Sherman           39
Winston               32
Maverick              28
305's                 25
Pyramid               19
marlboro              19
Sonoma                18
Salem                 14
USA Gold              12
Misty                 12
Dunhill                8
Basic                  1
Name: Brand, dtype: int64

In [16]:
# Variety
df_rating_5.Variety.value_counts()

Full Flavor                   521
Menthol                       331
Reds                          150
Classic                       116
Black                          95
Lights                         88
Black Menthol                  67
Filters Full Flavor            56
Smooth Menthol                 53
Turkish Royal                  47
Blend No. 27                   44
Luxury Lights                  42
Perique                        37
Wides Filters                  37
Crush                          36
Menthol Lights                 34
Lights (Blue)                  33
Special Blend (Black)          25
Straights                      24
Virginia Blend                 23
Lights (White)                 23
Filters                        23
Southern Cut                   21
Turkish Gold                   19
Menthol (Silver)               19
special blend                  19
Gold                           18
83's                           18
Black (Full Flavor)            17
72's          

In [17]:
# Type
df_rating_5.Type.value_counts()

Regular       1336
100s           715
Unfiltered     160
120s            54
Shorts          41
Menthol         11
Name: Type, dtype: int64

In [18]:
# Strength
df_rating_5.Strength.value_counts()

Strong         1018
Medium          868
Very Strong     346
Weak             71
Very Weak        14
Name: Strength, dtype: int64

In [19]:
# Taste
df_rating_5.Taste.value_counts()

Very Pleasant    1764
Pleasant          527
Tolerable          20
Very Poor           4
Poor                2
Name: Taste, dtype: int64

In [20]:
# Price
df_rating_5.Price.value_counts()

Fair         997
High         673
Very High    340
Low          219
Very Low      88
Name: Price, dtype: int64

In [21]:
# Query the highest frequency data in each column
df_pre_smoke = df_rating_5.query('Strength == "Strong" & Taste == "Very Pleasant" & Price == "Fair"')
df_pre_smoke

,User,Brand,Variety,Type,Date,Strength,Taste,Price,Rating
99,thomw,Camel,Filters Full Flavor,Regular,2007-07-12,Strong,Very Pleasant,Fair,5
120,homer676,Camel,Filters Full Flavor,Regular,2007-08-02,Strong,Very Pleasant,Fair,5
172,Rambus,Lucky Strike,Classic,Unfiltered,2007-08-16,Strong,Very Pleasant,Fair,5
180,Alex Umrysh,Camel,Filters Full Flavor,Regular,2007-08-20,Strong,Very Pleasant,Fair,5
202,Kaijapska,Camel,Filters Full Flavor,Regular,2007-08-29,Strong,Very Pleasant,Fair,5
...,...,...,...,...,...,...,...,...,...
5409,docmarlboro,Camel,Filters Full Flavor,Regular,2018-03-26,Strong,Very Pleasant,Fair,5
5423,Heike,L&M,Full Flavor,100s,2018-08-14,Strong,Very Pleasant,Fair,5
5424,Heike,L&M,Full Flavor,100s,2018-08-14,Strong,Very Pleasant,Fair,5
5427,Heike,Parliament,Full Flavor,Regular,2018-08-14,Strong,Very Pleasant,Fair,5


In [22]:
df_pre_smoke.nunique()

User        309
Brand        21
Variety      42
Type          5
Date        299
Strength      1
Taste         1
Price         1
Rating        1
dtype: int64

In [23]:
df_pre_smoke.Type.value_counts()

Regular       197
100s          111
Unfiltered     11
Shorts          6
120s            5
Name: Type, dtype: int64

In [24]:
# Query only 'Regular' type of cigarate
df_pre_smoke = df_pre_smoke.query('Type == "Regular"')
df_pre_smoke

,User,Brand,Variety,Type,Date,Strength,Taste,Price,Rating
99,thomw,Camel,Filters Full Flavor,Regular,2007-07-12,Strong,Very Pleasant,Fair,5
120,homer676,Camel,Filters Full Flavor,Regular,2007-08-02,Strong,Very Pleasant,Fair,5
180,Alex Umrysh,Camel,Filters Full Flavor,Regular,2007-08-20,Strong,Very Pleasant,Fair,5
202,Kaijapska,Camel,Filters Full Flavor,Regular,2007-08-29,Strong,Very Pleasant,Fair,5
287,EmphyzemaWOOHOO,Camel,Turkish Royal,Regular,2007-09-19,Strong,Very Pleasant,Fair,5
...,...,...,...,...,...,...,...,...,...
5323,NewfieSmoker,Camel,Crush,Regular,2017-06-30,Strong,Very Pleasant,Fair,5
5396,lil Donna,USA Gold,Full Flavor,Regular,2017-10-20,Strong,Very Pleasant,Fair,5
5406,TomChainsReds,Marlboro,Full Flavor,Regular,2018-03-11,Strong,Very Pleasant,Fair,5
5409,docmarlboro,Camel,Filters Full Flavor,Regular,2018-03-26,Strong,Very Pleasant,Fair,5


In [25]:
# create new dataframe
df_brand = df_pre_smoke.drop(['User','Type','Date','Strength','Taste','Price','Rating'], axis=1)
df_brand

,Brand,Variety
99,Camel,Filters Full Flavor
120,Camel,Filters Full Flavor
180,Camel,Filters Full Flavor
202,Camel,Filters Full Flavor
287,Camel,Turkish Royal
...,...,...
5323,Camel,Crush
5396,USA Gold,Full Flavor
5406,Marlboro,Full Flavor
5409,Camel,Filters Full Flavor


## Content-base Filtering

In [28]:
count = df_brand.groupby(['Brand','Variety']).size().reset_index(name = 'count')
count.sort_values(by='count', ascending = False)

,Brand,Variety,count
32,Marlboro,Reds,25
10,Camel,Filters Full Flavor,23
16,Camel,Wides Filters,12
34,Marlboro,Smooth Menthol,11
15,Camel,Turkish Royal,10
23,Marlboro,83's,9
35,Marlboro,Southern Cut,7
26,Marlboro,Blend No. 27,7
28,Marlboro,Full Flavor,7
17,Djarum,Black,5


In [29]:
# Query cigarette's brand and variety that picked >= 10 customers
count_10 = count.query('count >= 10')
count_10

,Brand,Variety,count
10,Camel,Filters Full Flavor,23
15,Camel,Turkish Royal,10
16,Camel,Wides Filters,12
32,Marlboro,Reds,25
34,Marlboro,Smooth Menthol,11


In [32]:
# Combine all conditons
# Create function that return recommended cigarette based on preference,
# 'rating', 'Strength', 'Taste', 'Price', 'Type'
def content_data(rating,Strength,Taste,Price,Type):
  data = df[df['Rating']== rating]
  data = data[(data['Strength'] == Strength)&(data['Taste'] == Taste)&(data['Price'] == Price)]
  data = data[data['Type'] == Type]
  data = data.drop(['User', 'Type', 'Date', 'Strength', 'Taste', 'Price', 'Rating'], axis=1)
  data = data.groupby(['Brand','Variety']).size().reset_index(name = 'count')
  data = data[data['count'] >= 10] # more than 10 people recommended these products
  return data

In [36]:
recommend = content_data(4,'Medium','Pleasant','Fair','Regular')
recommend

,Brand,Variety,count
5,Camel,Crush,14
8,Camel,Lights,11
15,Camel,Turkish Gold,12
28,Marlboro,Blend No. 27,10
38,Marlboro,Smooth Menthol,11


## Collaborative Filtering

This type of system matches user with similar interests and provides recommendations based on this matching

### Create User X Rating matrix

In [43]:
# create new column: Brand_variety
df['Brand_variety'] = df[['Brand','Variety']].apply(lambda x: '_'.join(x), axis=1)
df.head(10)

,User,Brand,Variety,Type,Date,Strength,Taste,Price,Rating,Brand_variety
0,ghanta,Marlboro,Reds,Regular,2007-06-21,Medium,Tolerable,Fair,3,Marlboro_Reds
1,Karawasa,Marlboro,Reds,Regular,2007-06-23,Very Strong,Tolerable,Fair,4,Marlboro_Reds
2,Tintedace,Camel,Turkish Gold,Regular,2007-06-23,Medium,Very Poor,High,1,Camel_Turkish Gold
3,Karawasa,Camel,Turkish Gold,Regular,2007-06-23,Strong,Pleasant,Fair,4,Camel_Turkish Gold
4,Tintedace,Newport,Full Flavor,Regular,2007-06-23,Medium,Very Pleasant,Fair,5,Newport_Full Flavor
5,Karawasa,Marlboro,Blend No. 27,Regular,2007-06-23,Medium,Pleasant,Fair,4,Marlboro_Blend No. 27
6,Grimey,Camel,No. 9,Regular,2007-06-23,Strong,Poor,Fair,2,Camel_No. 9
7,Grimey,Marlboro,Reds,Regular,2007-06-23,Very Strong,Poor,Fair,3,Marlboro_Reds
8,Karawasa,Parliament,Lights (White),Regular,2007-06-23,Medium,Tolerable,Fair,3,Parliament_Lights (White)
9,Grimey,Marlboro,Blend No. 27,Regular,2007-06-23,Weak,Poor,Fair,2,Marlboro_Blend No. 27


In [44]:
df[["User","Brand_variety","Rating"]]

,User,Brand_variety,Rating
0,ghanta,Marlboro_Reds,3
1,Karawasa,Marlboro_Reds,4
2,Tintedace,Camel_Turkish Gold,1
3,Karawasa,Camel_Turkish Gold,4
4,Tintedace,Newport_Full Flavor,5
...,...,...,...
5433,sahrah120s,Marlboro_Full Flavor,4
5434,sahrah120s,Marlboro_Black,4
5435,floras,Benson and Hedges_Gold,5
5436,Monijonnlopez,Marlboro_Special Blend (Red),5


In [45]:
# Matrix A: UserXFeature, Matrix B: FeatureXBrand_variety
# Matric C= Matrix A dot Matrix B
cmat = pd.crosstab(df['User'],df['Brand_variety'],values = df['Rating'],aggfunc='mean')
cmat

Brand_variety,305's_Full Flavor,American Spirit_Full Flavor,American Spirit_Lights,American Spirit_Mediums,American Spirit_Menthol,American Spirit_Menthol Lights,American Spirit_Organic,American Spirit_Perique,American Spirit_Ultra Lights,Basic_Full Flavor,...,Pyramid_Full Flavor,Salem_Green Label Full Flavor,Sonoma_Full Flavor,USA Gold_Full Flavor,Virginia Slims_Full Flavor,Virginia Slims_Luxury Lights,Virginia Slims_Menthol,Winston_Full Flavor,Winston_Lights,marlboro_special blend
User,,,,,,,,,,,,,,,,,,,,,
SavvLovesCamel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
!Eureka,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#1gramma,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
$ixtiesFreak,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00000,NaN,NaN,NaN,NaN,1.0,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zoey05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zoey0505,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zolimarsn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


NaN is blank data where users didn't rate some products

In [46]:
# fill NaN with 0
cmat = cmat.fillna(0)
cmat

Brand_variety,305's_Full Flavor,American Spirit_Full Flavor,American Spirit_Lights,American Spirit_Mediums,American Spirit_Menthol,American Spirit_Menthol Lights,American Spirit_Organic,American Spirit_Perique,American Spirit_Ultra Lights,Basic_Full Flavor,...,Pyramid_Full Flavor,Salem_Green Label Full Flavor,Sonoma_Full Flavor,USA Gold_Full Flavor,Virginia Slims_Full Flavor,Virginia Slims_Luxury Lights,Virginia Slims_Menthol,Winston_Full Flavor,Winston_Lights,marlboro_special blend
User,,,,,,,,,,,,,,,,,,,,,
SavvLovesCamel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
!Eureka,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#1gramma,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
$ixtiesFreak,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00000,0.0,0.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zoey05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zoey0505,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zolimarsn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
User = df['User'].unique()

In [49]:
# change User name to index
num_name_user = list(range(len(User)))
cmat.index = num_name_user
cmat

Brand_variety,305's_Full Flavor,American Spirit_Full Flavor,American Spirit_Lights,American Spirit_Mediums,American Spirit_Menthol,American Spirit_Menthol Lights,American Spirit_Organic,American Spirit_Perique,American Spirit_Ultra Lights,Basic_Full Flavor,...,Pyramid_Full Flavor,Salem_Green Label Full Flavor,Sonoma_Full Flavor,USA Gold_Full Flavor,Virginia Slims_Full Flavor,Virginia Slims_Luxury Lights,Virginia Slims_Menthol,Winston_Full Flavor,Winston_Lights,marlboro_special blend
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2948,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2949,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2950,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2951,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Decompose matix into two metrics using NMF

In [50]:
from sklearn.decomposition import NMF
# set feature = 30 (ลองกำหนดมาก่อนคร่าวๆ แล้วดูว่า lose เยอะมั้ย ถ้าเยอะก็เพิ่มจำนวน hidden feature)
nmf = NMF(n_components=30)
nmf.fit(cmat)

NMF(n_components=30)

In [51]:
nmf.components_.shape

(30, 92)

In [52]:
H = pd.DataFrame(np.round(nmf.components_,2), columns=cmat.columns)
H

Brand_variety,305's_Full Flavor,American Spirit_Full Flavor,American Spirit_Lights,American Spirit_Mediums,American Spirit_Menthol,American Spirit_Menthol Lights,American Spirit_Organic,American Spirit_Perique,American Spirit_Ultra Lights,Basic_Full Flavor,...,Pyramid_Full Flavor,Salem_Green Label Full Flavor,Sonoma_Full Flavor,USA Gold_Full Flavor,Virginia Slims_Full Flavor,Virginia Slims_Luxury Lights,Virginia Slims_Menthol,Winston_Full Flavor,Winston_Lights,marlboro_special blend
0,0.00,0.00,0.08,0.01,0.00,0.00,0.05,0.00,0.12,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
1,0.01,0.00,0.00,0.00,0.04,0.01,0.03,0.00,0.01,0.00,...,0.00,0.00,0.01,0.02,0.00,0.00,0.10,0.00,0.09,0.0
2,0.00,0.00,0.00,0.04,0.00,0.00,0.01,0.00,0.00,0.03,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.0
3,0.00,0.00,0.16,0.00,0.00,0.03,0.00,0.00,0.00,0.02,...,0.00,0.13,0.00,0.00,0.00,0.00,0.04,0.00,0.06,0.0
4,0.00,0.00,0.00,0.09,0.00,0.00,0.12,0.00,0.00,0.05,...,0.01,0.06,0.03,0.04,0.01,0.00,0.00,0.00,0.00,0.0
5,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.21,...,0.02,0.11,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
6,0.00,0.00,0.18,0.00,0.00,0.23,0.00,0.00,0.23,0.00,...,0.00,0.03,0.00,0.00,0.00,0.00,0.00,0.00,0.23,0.0
7,0.00,0.00,0.00,0.51,0.00,0.00,0.00,0.00,0.03,0.01,...,0.00,0.28,0.00,0.00,0.00,0.00,0.10,0.00,0.74,0.0
8,0.00,0.00,0.00,0.09,0.82,0.11,0.00,0.00,0.00,0.36,...,0.00,0.00,0.09,0.57,0.00,0.00,0.00,0.00,0.00,0.0
9,0.04,0.00,0.22,0.19,0.00,0.48,0.06,0.00,0.23,0.12,...,0.00,0.18,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0


In [53]:
W = pd.DataFrame(np.round(nmf.transform(cmat),2), columns=H.index)
W

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,...,0.0,0.00,0.00,0.56,0.00,0.00,0.00,0.0,0.00,0.00
1,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,...,0.0,0.00,0.01,0.00,0.00,0.19,0.00,0.0,0.02,0.00
2,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,...,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.12
3,0.0,0.0,0.0,0.00,0.0,0.0,0.01,0.00,0.00,0.01,...,0.0,0.02,0.03,0.00,0.00,0.00,0.04,0.0,0.02,0.00
4,0.0,0.0,0.0,0.47,0.0,0.0,0.01,0.00,0.01,0.01,...,0.0,0.04,0.02,0.00,0.00,0.00,0.05,0.0,0.00,0.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2948,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,...,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00
2949,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,...,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00
2950,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,...,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.44
2951,0.0,0.0,0.0,0.01,0.0,0.0,0.00,0.13,0.00,0.00,...,0.0,0.00,0.00,0.00,0.01,0.15,0.06,0.0,0.00,0.00


In [54]:
cmat

Brand_variety,305's_Full Flavor,American Spirit_Full Flavor,American Spirit_Lights,American Spirit_Mediums,American Spirit_Menthol,American Spirit_Menthol Lights,American Spirit_Organic,American Spirit_Perique,American Spirit_Ultra Lights,Basic_Full Flavor,...,Pyramid_Full Flavor,Salem_Green Label Full Flavor,Sonoma_Full Flavor,USA Gold_Full Flavor,Virginia Slims_Full Flavor,Virginia Slims_Luxury Lights,Virginia Slims_Menthol,Winston_Full Flavor,Winston_Lights,marlboro_special blend
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2948,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2949,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2950,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2951,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
reconstructed = pd.DataFrame(np.round(np.dot(W,H),2), columns=cmat.columns)
# W dot H => New table
reconstructed.index = cmat.index
reconstructed

Brand_variety,305's_Full Flavor,American Spirit_Full Flavor,American Spirit_Lights,American Spirit_Mediums,American Spirit_Menthol,American Spirit_Menthol Lights,American Spirit_Organic,American Spirit_Perique,American Spirit_Ultra Lights,Basic_Full Flavor,...,Pyramid_Full Flavor,Salem_Green Label Full Flavor,Sonoma_Full Flavor,USA Gold_Full Flavor,Virginia Slims_Full Flavor,Virginia Slims_Luxury Lights,Virginia Slims_Menthol,Winston_Full Flavor,Winston_Lights,marlboro_special blend
0,0.00,0.00,0.00,0.01,0.00,0.00,0.30,0.00,0.00,0.00,...,0.01,0.00,0.15,0.35,0.00,0.00,0.00,0.00,0.00,0.0
1,0.00,0.00,0.35,0.03,0.01,0.00,0.10,0.00,0.06,0.04,...,0.00,0.04,0.04,0.01,0.03,0.00,0.00,0.00,0.18,0.0
2,0.09,0.00,0.00,0.00,0.02,0.02,0.01,0.00,0.00,0.02,...,0.17,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.0
3,0.00,0.00,0.03,0.01,0.05,0.02,0.03,0.13,0.01,0.03,...,0.01,0.02,0.01,0.03,0.00,0.00,0.01,0.23,0.06,0.0
4,0.27,0.08,0.15,0.02,0.16,0.14,0.06,0.25,0.02,0.13,...,0.58,0.09,0.05,0.03,0.01,0.00,0.04,0.29,0.08,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2948,0.15,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
2949,0.15,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
2950,0.29,0.00,0.00,0.00,0.06,0.07,0.03,0.00,0.00,0.08,...,0.64,0.00,0.02,0.00,0.00,0.00,0.02,0.00,0.00,0.0
2951,0.00,0.08,0.28,0.09,0.01,0.00,0.08,0.00,0.04,0.03,...,0.00,0.07,0.01,0.03,0.03,0.08,0.02,0.34,0.21,0.0


User1

- Before reconstructed: American Spirit Mental = 0
- After reconstructed: American Spirit Metal = 0.01 => เพิ่มโอกาสในการแนะนำสินค้านี้ ทั้งที่ user ไม่เคยรู้จักหรือใช้มาก่อน

In [56]:
# filter only user1
reconstructed.T #transpose table
#user=1
# topk => return top 10 products
# no need to tell user about score tell only product
reconstructed.T[1].sort_values(ascending=False)[0:10]

Brand_variety
Pall Mall_Lights (Blue)      4.06
Parliament_Lights (White)    1.06
Camel_Lights                 0.78
American Spirit_Lights       0.35
Marlboro_Virginia Blend      0.33
Marlboro_Menthol Lights      0.24
Pall Mall_Reds               0.23
Marlboro_Menthol Milds       0.18
Winston_Lights               0.18
L&M_Lights                   0.17
Name: 1, dtype: float64

In [58]:
# create function to return recommended products based on each user
def recommendation(uid,topk=5):
  res = reconstructed.T[uid].sort_values(ascending=False)[0:topk]
  # filter score > 0
  res = list(res[res>0].index)
  res = df[df['Brand_variety'].isin(res)]
  res = res.drop_duplicates(subset='Brand_variety')
  res = res[:topk]
  res = res[['Brand_variety','Strength','Taste','Price']]
  return res

In [59]:
# assumer our target user is user1
recommendation(1,topk=10)

,Brand_variety,Strength,Taste,Price
8,Parliament_Lights (White),Medium,Tolerable,Fair
16,Camel_Lights,Medium,Very Pleasant,Low
18,American Spirit_Lights,Medium,Pleasant,High
20,Winston_Lights,Weak,Poor,Fair
26,Marlboro_Menthol Lights,Weak,Poor,Fair
40,Marlboro_Menthol Milds,Medium,Pleasant,Low
137,Pall Mall_Lights (Blue),Weak,Tolerable,Fair
325,Marlboro_Virginia Blend,Strong,Very Pleasant,High
326,L&M_Lights,Medium,Very Pleasant,Fair
1865,Pall Mall_Reds,Medium,Very Pleasant,Fair
